In [1]:
import peachbox

In [2]:
batch_views = peachbox.CassandraDriver()
batch_views.set_keyspace('batch_views')

In [3]:
query = batch_views.execute("SELECT * FROM reviews")

In [4]:
list(query)[:10]

[Row(partition_key=u'00b92251-1606-de11-e588-950242ac1100', product_id=u'B00004CMYH', review_count=1, review_id=u'7412817493041903523', score=5, true_as_of_seconds=915321600),
 Row(partition_key=u'474c6cc4-06df-11e5-8895-0242ac110002', product_id=u'630460288X', review_count=36, review_id=u'-7807804278603516210', score=5, true_as_of_seconds=898214400),
 Row(partition_key=u'00e0bf20-6a06-dd11-e588-950242ac1100', product_id=u'6304674732', review_count=36, review_id=u'-4118651289483639760', score=3, true_as_of_seconds=893116800),
 Row(partition_key=u'00b74264-e406-de11-e588-950242ac1100', product_id=u'6303997643', review_count=1, review_id=u'-2657806680723346140', score=1, true_as_of_seconds=917222400),
 Row(partition_key=u'281f4f8e-06de-11e5-8895-0242ac110002', product_id=u'B000ICZD5C', review_count=4, review_id=u'-2967733849407159224', score=4, true_as_of_seconds=914198400),
 Row(partition_key=u'00ffad49-ba06-de11-e588-950242ac1100', product_id=u'B000F6R048', review_count=1, review_id=u'

ERROR:cassandra.metadata:Tried to update schema for table 'reviews' in unknown keyspace 'real_time_views'


In [4]:
query = batch_views.execute("SELECT true_as_of_seconds, review_count FROM reviews WHERE product_id='B0002KPI2S'")
review_counts = map(lambda x: (x.true_as_of_seconds, x.review_count), query)
review_counts                                                                              

[(1432998000, 1),
 (1432998000, 1),
 (1432998000, 1),
 (1432998000, 1),
 (911520000, 36),
 (915580800, 4),
 (911520000, 36),
 (1432998000, 1),
 (1432998000, 1),
 (911520000, 36),
 (915580800, 4),
 (1432998000, 1),
 (920937600, 1),
 (1433840400, 1),
 (920937600, 1),
 (1432998000, 1),
 (915580800, 4),
 (1432998000, 1),
 (1432998000, 1),
 (920937600, 1),
 (1432998000, 1),
 (911520000, 36),
 (1432998000, 1),
 (920937600, 1),
 (915580800, 4),
 (1432998000, 1),
 (1433840400, 1),
 (1432998000, 1),
 (911520000, 36),
 (1432998000, 1),
 (920937600, 1),
 (915580800, 4)]

In [5]:
latest_timestamp = max(map(lambda x: x[0], review_counts))
latest_timestamp

1433840400

In [6]:
real_time_views = peachbox.CassandraDriver()
real_time_views.set_keyspace('real_time_views')
real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='B0002KPI2S'\
                            AND true_as_of_seconds>"+str(latest_timestamp)+" ALLOW FILTERING")

[Row(count=1)]

In [13]:
real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='B0002KPI2S'")

[Row(count=18)]

In [12]:
import kafka.client                                                                                                        
import kafka.producer      
import ujson as json
import time
                                                                                                                           
client   = kafka.client.KafkaClient('localhost:9092')                                                                      
producer = kafka.producer.SimpleProducer(client)                                                                           
                                                                                                                           
message = {'user_id':'A37I5QIB*9UMPD',
           'product_id':'B0002KPI2S',
           'review':'review text',
           'summary':'Summarizing a great movie',
           'profile_name':'ItsMe',
           'helpfulness':'2/2',
           'time':int(time.time()),
           'score':5.0}
                                                                                                   
producer.send_messages('movie_reviews', json.dumps(message))                                                                      


[ProduceResponse(topic='movie_reviews', partition=0, error=0, offset=401208)]

In [25]:
def review_count(product_id):
    batch_views = peachbox.CassandraDriver()
    batch_views.set_keyspace('batch_views')
    
    real_time_views = peachbox.CassandraDriver()
    real_time_views.set_keyspace('real_time_views')

    query = batch_views.execute("SELECT true_as_of_seconds, review_count\
                                 FROM reviews WHERE product_id='"+product_id+"'")
    review_counts    = map(lambda x: (x.true_as_of_seconds, x.review_count), query)
    latest_timestamp = max(map(lambda x: x[0], review_counts))
    rt_review_counts = real_time_views.execute("SELECT COUNT(*) FROM reviews WHERE product_id='"+product_id+"'\
                            AND true_as_of_seconds>"+str(latest_timestamp)+" ALLOW FILTERING")[0].count
    return sum(map(lambda x: x[1], review_counts)) + rt_review_counts

In [26]:
review_count('B00004U12X')

15